<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [ ]:
import pandas as pd
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras import layers
import keras
! pip install -q -U keras-tuner
import keras_tuner as kt


# 1. 모든 피험자 파일 리스트 가져오기
files = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET/mHealth_subject*.log"))

# 2. 컬럼 이름 정의 (센서 feature 23개 + label)
columns = [
    "chest_acc_x", "chest_acc_y", "chest_acc_z",
    "ecg_1", "ecg_2",
    "chest_gyro_x", "chest_gyro_y", "chest_gyro_z",
    "chest_mag_x", "chest_mag_y", "chest_mag_z",
    "ankle_acc_x", "ankle_acc_y", "ankle_acc_z",
    "ankle_gyro_x", "ankle_gyro_y", "ankle_gyro_z",
    "ankle_mag_x", "ankle_mag_y", "ankle_mag_z",
    "wrist_acc_x", "wrist_acc_y", "wrist_acc_z",
    "label"
]

# 3. 각 파일 읽어서 subject_id 추가
dfs = []
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file, sep="\s+", header=None)  # 공백 구분자로 읽기
    df.columns = columns
    df["subject_id"] = i  # 피험자 번호 추가
    dfs.append(df)

# 4. 모든 피험자 데이터 통합
data = pd.concat(dfs, ignore_index=True)

# 5. 확인
print(data.head())
print(data.shape)



<>:34: SyntaxWarning: invalid escape sequence '\s'
<>:34: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-429090591.py:34: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep="\s+", header=None)  # 공백 구분자로 읽기


   chest_acc_x  chest_acc_y  chest_acc_z     ecg_1     ecg_2  chest_gyro_x  \
0      -9.8184     0.009971      0.29563  0.004186  0.004186        2.1849   
1      -9.8489     0.524040      0.37348  0.004186  0.016745        2.3876   
2      -9.6602     0.181850      0.43742  0.016745  0.037677        2.4086   
3      -9.6507     0.214220      0.24033  0.079540  0.117220        2.1814   
4      -9.7030     0.303890      0.31156  0.221870  0.205130        2.4173   

   chest_gyro_y  chest_gyro_z  chest_mag_x  chest_mag_y  ...  ankle_gyro_y  \
0       -9.6967       0.63077     0.103900     -0.84053  ...       -4.5781   
1       -9.5080       0.68389     0.085343     -0.83865  ...       -4.3198   
2       -9.5674       0.68113     0.085343     -0.83865  ...       -4.2772   
3       -9.4301       0.55031     0.085343     -0.83865  ...       -4.3163   
4       -9.3889       0.71098     0.085343     -0.83865  ...       -4.1459   

   ankle_gyro_z  ankle_mag_x  ankle_mag_y  ankle_mag_z  wrist_

In [ ]:
data

,chest_acc_x,chest_acc_y,chest_acc_z,ecg_1,ecg_2,chest_gyro_x,chest_gyro_y,chest_gyro_z,chest_mag_x,chest_mag_y,...,ankle_gyro_y,ankle_gyro_z,ankle_mag_x,ankle_mag_y,ankle_mag_z,wrist_acc_x,wrist_acc_y,wrist_acc_z,label,subject_id
0,-9.8184,0.009971,0.295630,0.004186,0.004186,2.18490,-9.6967,0.63077,0.103900,-0.84053,...,-4.5781,0.187760,-0.44902,-1.01030,0.034483,-2.35000,-1.610200,-0.030899,0,1
1,-9.8489,0.524040,0.373480,0.004186,0.016745,2.38760,-9.5080,0.68389,0.085343,-0.83865,...,-4.3198,0.023595,-0.44902,-1.01030,0.034483,-2.16320,-0.882540,0.326570,0,1
2,-9.6602,0.181850,0.437420,0.016745,0.037677,2.40860,-9.5674,0.68113,0.085343,-0.83865,...,-4.2772,0.275720,-0.44902,-1.01030,0.034483,-1.61750,-0.165620,-0.030693,0,1
3,-9.6507,0.214220,0.240330,0.079540,0.117220,2.18140,-9.4301,0.55031,0.085343,-0.83865,...,-4.3163,0.367520,-0.45686,-1.00820,0.025862,-1.07710,0.006945,-0.382620,0,1
4,-9.7030,0.303890,0.311560,0.221870,0.205130,2.41730,-9.3889,0.71098,0.085343,-0.83865,...,-4.1459,0.407290,-0.45686,-1.00820,0.025862,-0.53684,0.175900,-1.095500,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215740,-12.2440,-2.970600,-0.035772,0.812140,1.096800,0.57061,-2.5602,0.41936,-0.055659,0.64165,...,-16.9660,2.370400,0.10980,-0.99384,0.191810,-6.12870,15.495000,18.856000,0,10
1215741,-10.9220,-3.419000,-0.339280,1.469400,1.285200,5.26840,-4.9331,9.66020,-0.055659,0.64165,...,-13.3900,2.997200,0.10980,-0.99384,0.191810,-10.24200,17.139000,33.920000,0,10
1215742,-9.4842,-3.064300,-1.033700,0.238620,0.891680,0.53805,-5.9706,2.93600,-0.055659,0.64165,...,-11.3790,3.149800,0.10980,-0.99384,0.191810,-12.94400,16.170000,43.262000,0,10
1215743,-8.7889,-2.475700,-0.612290,-0.205130,0.460490,1.56950,-7.9809,-2.05000,-0.085343,0.43715,...,-10.0000,3.000000,0.11961,-0.97331,0.153020,-14.52400,1.849400,43.373000,0,10


In [ ]:
from collections.abc import Sequence
import numpy as np

Sequence=100
n_steps=50
# 슬라이딩 윈도우 함수
def split_sequences(sequences, n_steps):
    X, y = [], []
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        X.append(sequences[i:end_ix, :-1])
        y.append(sequences[end_ix-1, -1])
    return np.array(X), np.array(y)

In [ ]:
# Scale train/ test predictors based on training data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = keras.models.Sequential()
model.add(keras.Input(shape=(23,23))) # seq_length, input_size
model.add(layers.SimpleRNN(256, return_sequences=True, activation='relu')) # N, 28, 128
model.add(layers.SimpleRNN(256, return_sequences=False, activation='relu')) # N, 128
model.add(layers.Dense(52))
model.add(layers.Dense(13))
print(model.summary())

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_4 (SimpleRNN)        │ (None, 23, 256)        │        71,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 52)             │        13,364 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 13)             │           689 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 217,061 (847.89 KB)

 Trainable params: 217,061 (847.89 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
callbacks = [keras.callbacks.ModelCheckpoint("mhealth_best.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)]

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["sparse_categorical_accuracy"],) # Changed loss to categorical_crossentropy

model_history = model.fit(X_train, y_train, epochs= 10, validation_data=(X_test, y_test), callbacks=callbacks) # Used sequence and one-hot encoded data

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 23), dtype=float32). Expected shape (None, 23, 23), but input has incompatible shape (None, 23)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 23), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>